## Import Libraries

In [97]:
import pandas as pd
import plotly.express as px
import plotly

## Load Data

In [98]:
with open('data/headchopper.txt') as f:
    data = f.read()

data = data.split('\n')
data = pd.DataFrame(data)
print(len(data))
data.head()

3968


,0
0,"[9/3/23, 00:30:58] Headchopper: ‎Messages and ..."
1,"[9/3/23, 00:30:58] Headchopper: Hey Cesar! My ..."
2,"[9/3/23, 00:32:43] Cesar: Heyy"
3,"[9/3/23, 00:32:52] Cesar: Come"
4,"[9/3/23, 00:32:55] Cesar: Diego de leon 40"


## Clean Data

In [100]:
# join messages that are split into multiple rows
for i in range(1, len(data)):
    if data.iloc[i].values[0].startswith('['):
        continue
        #print(data.iloc[i].values[0])
    #Im not convinced about this while loop. It seems to be working but I'm not sure if it's the best way to do it
    while not data.iloc[i].values[0].startswith('['):
        data.iloc[i-1] = data.iloc[i-1].values[0] + ' ' + data.iloc[i].values[0]
        data = data.drop(i)
        data = data.reset_index(drop=True)


IndexError: single positional indexer is out-of-bounds

In [101]:
# truncate messages that are too long
for i in range(0, len(data)):
    message = data.iloc[i].values[0]
    if len(message) > 500:
        print(message[:500])
        data.iloc[i] = message[:500]

[11/8/23, 08:38:10] Headchopper: 🚀 Hi everyone! We hope your semester has been going well. We are so excited to announce our next event!  🤖 Unleash the Power of Neural Networks!  Calling all tech enthusiasts, from beginners to seasoned pros! Get ready to embark on an exhilarating journey into the world of neural networks at our upcoming event hosted by the Google Developer Students Club.  🌟 What to Expect:  🧠 Demystifying Neural Networks: Dive headfirst into the fascinating realm of artificial i
[1/17/24, 09:53:06] Headchopper: Actually my roommate is sleeping and I don’t wanna wake her 😂  Warsaw- definitely good if you want a nice quick stop over in Poland, ideal mix between city, ww2 history, beautiful parks, fun nightlife, farmers markets, beautiful & cozy old town. A lot of the old town was rebuilt though cause it was absolutely destroyed in ww2.   Krakow is cheaper, has a more well-preserved medieval old town. I haven’t been here since I was a kid!   Sopot/Gdansk are smaller towns

In [102]:
# If it starts with ‎, remove the row
data = data[~data[0].str.startswith('‎')]

In [103]:
#look at the rows between 120 and 130
data[117:118].values

array([['[11/8/23, 08:38:10] Headchopper: 🚀 Hi everyone! We hope your semester has been going well. We are so excited to announce our next event!  🤖 Unleash the Power of Neural Networks!  Calling all tech enthusiasts, from beginners to seasoned pros! Get ready to embark on an exhilarating journey into the world of neural networks at our upcoming event hosted by the Google Developer Students Club.  🌟 What to Expect:  🧠 Demystifying Neural Networks: Dive headfirst into the fascinating realm of artificial i']],
      dtype=object)

In [104]:
# total number of messages
len(data)

3618

## Dates

In [106]:
sp_data = data.copy()
sp_data['date'] = sp_data[0].str.extract(r'\[(\d+/\d+/\d+, \d+:\d+:\d+)\]')
sp_data['date'] = pd.to_datetime(sp_data['date'], format="%m/%d/%y, %H:%M:%S")
sp_data['day'] = sp_data['date'].dt.date.astype('str')
sp_data['hour'] = sp_data['date'].dt.hour

sp_data.head()

,0,date,day,hour
0,"[9/3/23, 00:30:58] Headchopper: ‎Messages and ...",2023-09-03 00:30:58,2023-09-03,0
1,"[9/3/23, 00:30:58] Headchopper: Hey Cesar! My ...",2023-09-03 00:30:58,2023-09-03,0
2,"[9/3/23, 00:32:43] Cesar: Heyy",2023-09-03 00:32:43,2023-09-03,0
3,"[9/3/23, 00:32:52] Cesar: Come",2023-09-03 00:32:52,2023-09-03,0
4,"[9/3/23, 00:32:55] Cesar: Diego de leon 40",2023-09-03 00:32:55,2023-09-03,0


## Messages

In [107]:
dt_data = sp_data.copy()
dt_data['sender'] = sp_data[0].str.extract(r'\[\d+/\d+/\d+, \d+:\d+:\d+\] ([a-zA-z]+):.*')
dt_data['message'] = sp_data[0].str.extract(r'\[\d+/\d+/\d+, \d+:\d+:\d+\] [a-zA-z]+:(.*)')
dt_data.head(10)

,0,date,day,hour,sender,message
0,"[9/3/23, 00:30:58] Headchopper: ‎Messages and ...",2023-09-03 00:30:58,2023-09-03,0,Headchopper,‎Messages and calls are end-to-end encrypted....
1,"[9/3/23, 00:30:58] Headchopper: Hey Cesar! My ...",2023-09-03 00:30:58,2023-09-03,0,Headchopper,Hey Cesar! My event is wrapping up and I’m th...
2,"[9/3/23, 00:32:43] Cesar: Heyy",2023-09-03 00:32:43,2023-09-03,0,Cesar,Heyy
3,"[9/3/23, 00:32:52] Cesar: Come",2023-09-03 00:32:52,2023-09-03,0,Cesar,Come
4,"[9/3/23, 00:32:55] Cesar: Diego de leon 40",2023-09-03 00:32:55,2023-09-03,0,Cesar,Diego de leon 40
5,"[9/3/23, 00:32:57] Cesar: 4B",2023-09-03 00:32:57,2023-09-03,0,Cesar,4B
6,"[9/3/23, 00:35:25] Headchopper: Okay cool! I’l...",2023-09-03 00:35:25,2023-09-03,0,Headchopper,Okay cool! I’ll text before I come to make su...
7,"[9/3/23, 00:37:20] Cesar: Nice",2023-09-03 00:37:20,2023-09-03,0,Cesar,Nice
8,"[9/11/23, 21:43:36] Cesar: I bet you are no lo...",2023-09-11 21:43:36,2023-09-11,21,Cesar,I bet you are no longer liking the idea of ho...
9,"[9/11/23, 21:43:48] Cesar: Lets find a place n...",2023-09-11 21:43:48,2023-09-11,21,Cesar,Lets find a place nearby we can all fit


In [108]:
dt_data['sender'] = dt_data['sender'].str.strip()
dt_data['message'] = dt_data['message'].str.strip()
dt_data.head(10)

,0,date,day,hour,sender,message
0,"[9/3/23, 00:30:58] Headchopper: ‎Messages and ...",2023-09-03 00:30:58,2023-09-03,0,Headchopper,‎Messages and calls are end-to-end encrypted. ...
1,"[9/3/23, 00:30:58] Headchopper: Hey Cesar! My ...",2023-09-03 00:30:58,2023-09-03,0,Headchopper,Hey Cesar! My event is wrapping up and I’m thi...
2,"[9/3/23, 00:32:43] Cesar: Heyy",2023-09-03 00:32:43,2023-09-03,0,Cesar,Heyy
3,"[9/3/23, 00:32:52] Cesar: Come",2023-09-03 00:32:52,2023-09-03,0,Cesar,Come
4,"[9/3/23, 00:32:55] Cesar: Diego de leon 40",2023-09-03 00:32:55,2023-09-03,0,Cesar,Diego de leon 40
5,"[9/3/23, 00:32:57] Cesar: 4B",2023-09-03 00:32:57,2023-09-03,0,Cesar,4B
6,"[9/3/23, 00:35:25] Headchopper: Okay cool! I’l...",2023-09-03 00:35:25,2023-09-03,0,Headchopper,Okay cool! I’ll text before I come to make sur...
7,"[9/3/23, 00:37:20] Cesar: Nice",2023-09-03 00:37:20,2023-09-03,0,Cesar,Nice
8,"[9/11/23, 21:43:36] Cesar: I bet you are no lo...",2023-09-11 21:43:36,2023-09-11,21,Cesar,I bet you are no longer liking the idea of hos...
9,"[9/11/23, 21:43:48] Cesar: Lets find a place n...",2023-09-11 21:43:48,2023-09-11,21,Cesar,Lets find a place nearby we can all fit


## Cleaned


In [109]:
dt_data = dt_data.drop(columns=0)
dt_data.head(10)

,date,day,hour,sender,message
0,2023-09-03 00:30:58,2023-09-03,0,Headchopper,‎Messages and calls are end-to-end encrypted. ...
1,2023-09-03 00:30:58,2023-09-03,0,Headchopper,Hey Cesar! My event is wrapping up and I’m thi...
2,2023-09-03 00:32:43,2023-09-03,0,Cesar,Heyy
3,2023-09-03 00:32:52,2023-09-03,0,Cesar,Come
4,2023-09-03 00:32:55,2023-09-03,0,Cesar,Diego de leon 40
5,2023-09-03 00:32:57,2023-09-03,0,Cesar,4B
6,2023-09-03 00:35:25,2023-09-03,0,Headchopper,Okay cool! I’ll text before I come to make sur...
7,2023-09-03 00:37:20,2023-09-03,0,Cesar,Nice
8,2023-09-11 21:43:36,2023-09-11,21,Cesar,I bet you are no longer liking the idea of hos...
9,2023-09-11 21:43:48,2023-09-11,21,Cesar,Lets find a place nearby we can all fit


# Data Exploration

In [110]:
dt_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3618 entries, 0 to 3617
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   date     3618 non-null   datetime64[ns]
 1   day      3618 non-null   object        
 2   hour     3618 non-null   int32         
 3   sender   3618 non-null   object        
 4   message  3618 non-null   object        
dtypes: datetime64[ns](1), int32(1), object(3)
memory usage: 127.3+ KB


In [111]:
dt_data['sender'].value_counts()

sender
Headchopper    1865
Cesar          1753
Name: count, dtype: int64

In [112]:
top_10 = dt_data['sender'].value_counts()
#plot a pie chart
fig = px.pie(values=top_10, names=top_10.index, title='Number of messages sent by each user')
fig.show()


In [113]:
#plot the date and time of the messages per day
fig = px.bar(dt_data['date'].dt.date.value_counts(), title='Number of messages per day')
fig.update_layout(xaxis_title='Date', yaxis_title='Number of messages')
fig.show()


In [114]:
#mask = dt_data['day'] == '2023-12-15'
#dt_data[mask]

In [115]:
#plot the number of messages per hour
fig = px.bar(dt_data['hour'].value_counts(), title='Number of messages per hour')
fig.update_layout(xaxis_title='Hour', yaxis_title='Number of messages')
fig.show()

# Sentiment Analysis

In [14]:
from transformers import pipeline

/Users/cesargarcia/anaconda3/envs/pda-ie/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning:

torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.



In [17]:
sent_pipeline = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [84]:
example = dt_data['message'].tolist()[1]
print(example)
sent_pipeline(example)

Hey Cesar! My event is wrapping up and I’m thinking of joining you guys- what is your address for the pre drinks?


[{'label': 'POSITIVE', 'score': 0.9977076053619385}]

In [85]:
%%time
#print number of messages
total_messages = len(dt_data['message'])
print(f'Total of {total_messages} messages')

#Apply the sentiment analysis to all the messages
messages = dt_data['message'].tolist()
for index, message in enumerate(messages):
    if type(message) != str:
        print('Not a string')
        print(f"Index: {index}")
        
sentiments = sent_pipeline(messages)

#Add the sentiment to the dataframe
dt_data['sentiment'] = [sent['label'] for sent in sentiments]
dt_data['score'] = [sent['score'] for sent in sentiments]
dt_data.head()

Total of 3618 messages


Token indices sequence length is longer than the specified maximum sequence length for this model (1450 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (1450) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
avr_sentiments = []
for i in range(0, len(sentiments)):
    avr_sentiments.append(sentiments[i]['label'])

avr_sentiments = pd.DataFrame(avr_sentiments)
avr_sentiments['sender'] = dt_data['sender'].value_counts().index[0]